In [1]:
from animal_soup import *

2023-05-29 15:57:14.703190: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-29 15:57:15.151973: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/clewis7/repos/animal-soup/animal_soup/batch_utils.py:75: UserWarning: registration of accessor <class 'animal_soup.batch_utils.PathsDataFrameExtension'> under name 'paths' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  @pd.api.extensions.register_dataframe_accessor("paths")
/home/clewis7/repos/animal-soup/animal_soup/batch_utils.py:80: UserWarning: registration of accessor <class 'animal_soup.batch_utils.PathsSeriesExtension'> under name 'paths' for type <class 'pandas.core.series.Series'> is 

In [2]:
training_data_path = set_parent_raw_data_path('/data/caitlin/training-vids/')
df_path = '/data/caitlin/data-frames/deepetho_results.hdf'

In [3]:
df = load_df(df_path)

In [4]:
df

,animal_id,session_id,mat_file,session_vids,notes
0,M235Slc17a7_Gtacr2,20170424,/data/caitlin/training-vids/M235Slc17a7_Gtacr2...,[/data/caitlin/training-vids/M235Slc17a7_Gtacr...,None
1,M235Slc17a7_Gtacr2,20170425,/data/caitlin/training-vids/M235Slc17a7_Gtacr2...,[/data/caitlin/training-vids/M235Slc17a7_Gtacr...,None
2,M236Slc17a7_Gtacr2,20170501,/data/caitlin/training-vids/M236Slc17a7_Gtacr2...,[/data/caitlin/training-vids/M236Slc17a7_Gtacr...,None


In [5]:
container = df.behavior.view(deg_comparison=True)

/home/clewis7/venvs/mescore/lib/python3.11/site-packages/ipydatagrid/datagrid.py:445: UserWarning: Index name of 'index' is not round-trippable.
  schema = pd.io.json.build_table_schema(dataframe)


RFBOutputContext()

/home/clewis7/repos/mesmerize-core/mesmerize_core/arrays/_video.py:87: UserWarning: min not implemented for LazyTiff, returning min of 0th index
  warn("min not implemented for LazyTiff, returning min of 0th index")
/home/clewis7/repos/mesmerize-core/mesmerize_core/arrays/_video.py:92: UserWarning: max not implemented for LazyTiff, returning min of 0th index
  warn("max not implemented for LazyTiff, returning min of 0th index")


RFBOutputContext()

RFBOutputContext()

In [7]:
container.show()

In [17]:
from pathlib import Path

In [18]:
deg_data_path = Path('/data/caitlin/deg-jaaba/DATA')

In [22]:
import pandas as pd
import numpy as np
import os

In [ ]:
def save_deg_prediction(deg_data_path, animal_id, session_id):
    preds_not_saved = list()
    row = df.loc[(df['animal_id'] == 'M236Slc17a7_Gtacr2') & (df['session_id'] == '20170501') ]
    options = [item.stem for item in row['session_vids'].item()]
    animal_id = row['animal_id'].item()
    session_id = row['session_id'].item()
    for op in options:
        # get the labels file
        try:
            deg_prediction = pd.read_csv(filepath_or_buffer=deg_data_path.joinpath(op).joinpath(op).with_name(f'{op}_predictions').with_suffix('.csv'))
            deg_prediction = deg_prediction.drop('background', axis=1)
            deg_prediction = deg_prediction.drop('Unnamed: 0', axis=1)
            behaviors = deg_prediction.columns
            deg_prediction = deg_prediction.to_numpy(dtype='uint8').T
            np.save(training_data_path.joinpath(animal_id).joinpath(session_id).joinpath(session_id).with_name(f'{op}').with_suffix('.npy'), deg_prediction)
        except FileNotFoundError:
            preds_not_saved.append(op)
            continue

In [75]:
from cv2 import EMD, DIST_L2

In [76]:
def emd_1d(u: np.ndarray, v: np.ndarray) -> float:
    """
    Wrapper around OpenCV Earth Mover's Distance (EMD) function to calculate EMD between 1D arrays
    :param u: an array of weights
    :param v: an array of weights
    :return: Earth Mover's Distance between u & v
    """

    a = np.empty((u.size, 2), dtype=np.float32)
    b = np.empty((v.size, 2), dtype=np.float32)

    a[:, 0] = u.astype(np.float32)
    a[:, 1] = np.arange(u.size, dtype=np.float32)

    b[:, 0] = v.astype(np.float32)
    b[:, 1] = np.arange(v.size, dtype=np.float32)

    return EMD(a, b, DIST_L2)[0]

In [78]:
emds = list()
for jab, deg in zip(jab_ethogram_array, deg_ethogram_array):
    emds.append(emd_1d(jab, deg))

In [79]:
emds

[0.1794871836900711,
 62.82143020629883,
 0.0,
 0.8217821717262268,
 7.82608699798584,
 7.370370388031006]